# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()
print(collections)

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)

# Display the first document in the results using pprint
first_hygiene_20 = establishments.find_one(query)
pprint(first_hygiene_20)

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67560a940798e8f6ababc2ac'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate

In [7]:
# Convert the result to a Pandas DataFrame
hygiene_20_results = establishments.find(query)
hygiene_20_df = pd.DataFrame(list(hygiene_20_results))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {hygiene_20_df.shape[0]}")

# Display the first 10 rows of the DataFrame
print(hygiene_20_df.head(10))

Number of rows in the DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  67560a940798e8f6ababc2ac   110681                  0   
1  67560a940798e8f6ababc632   612039                  0   
2  67560a940798e8f6ababc941   730933                  0   
3  67560a940798e8f6ababcb2a   172735                  0   
4  67560a940798e8f6ababcb36   172953                  0   
5  67560a940798e8f6ababd4db   512854                  0   
6  67560a940798e8f6ababd6fd  1537089                  0   
7  67560a950798e8f6ababec24   155648                  0   
8  67560a950798e8f6ababf06b  1012883                  0   
9  67560a950798e8f6ababf873   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [13]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
london_rating_count = establishments.count_documents(query)
print(f"Number of establishments in London with RatingValue >= 4: {london_rating_count}")

# Display the first document in the results using pprint
first_london_rating = establishments.find_one(query)
pprint(first_london_rating)

Number of establishments in London with RatingValue >= 4: 0
None


In [14]:
# Convert the result to a Pandas DataFrame
london_rating_results = establishments.find(query)

# Display the number of rows in the DataFrame
london_rating_df = pd.DataFrame(list(london_rating_results))

# Display the first 10 rows of the DataFrame
print(f"Number of rows in the DataFrame: {london_rating_df.shape[0]}")
print(london_rating_df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [15]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000


query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - 0.01, "$lte": latitude + 0.01},
    "geocode.longitude": {"$gte": longitude - 0.01, "$lte": longitude + 0.01}
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
top_5_nearest = establishments.find(query).sort(sort).limit(limit)
top_5_df = pd.DataFrame(list(top_5_nearest))

In [16]:
# Convert result to Pandas DataFrame
print(f"Top 5 nearest establishments with RatingValue 5, sorted by hygiene score:")
print(top_5_df)

Top 5 nearest establishments with RatingValue 5, sorted by hygiene score:
                        _id   FHRSID  ChangesByServerID  \
0  67560a950798e8f6abac1b08  1380578                  0   
1  67560a950798e8f6abac1b2d   694478                  0   
2  67560a950798e8f6abac1b26   695241                  0   
3  67560a950798e8f6abac1ae6   695223                  0   
4  67560a950798e8f6abac1b0f   694609                  0   

  LocalAuthorityBusinessID                         BusinessName  \
0                    14425  Howe and Co Fish and Chips - Van 17   
1             PI/000086506                    Atlantic Fish Bar   
2             PI/000179088              Plumstead Manor Nursery   
3             PI/000178842                              Iceland   
4             PI/000116619                            Volunteer   

                            BusinessType  BusinessTypeID  \
0                         Mobile caterer            7846   
1                 Takeaway/sandwich shop        

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [21]:
# Create a pipeline that:
pipeline = [

# 1. Matches establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
# 2. Groups the matches by Local Authority
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
# 3. Sorts the matches from highest to lowest
    {"$sort": {"count": -1}}
]   
hygiene_0_results = establishments.aggregate(pipeline)

# Print the number of documents in the result
hygiene_0_list = list(hygiene_0_results)
print(f"Number of documents in the result: {len(hygiene_0_list)}")

# Print the first 10 results
print("First 10 results:")
for result in hygiene_0_list[:10]:
    print(result)


Number of documents in the result: 55
First 10 results:
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [22]:
# Convert the result to a Pandas DataFrame
hygiene_0_df = pd.DataFrame(hygiene_0_list)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {hygiene_0_df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(hygiene_0_df.head(10))

Number of rows in the DataFrame: 55

First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
